# Chapter 1 - Introduction

Link to "m-file": http://people.maths.ox.ac.uk/trefethen/ExplODE/ode01.m

## Environment setup
Setup the local stuff - first you may need to change the Kernel, but not always.

Then setup the packages as needed. The stuff that's commented out should already be there from the build. Don't actually load that unless you need to.


In [ ]:
using Pkg
Pkg.status()

# Pkg.add("DifferentialEquations")
# Pkg.add("Plots")
# Pkg.add("DiffEqBase")
# Pkg.add("DiffEqOperators")
# Pkg.add("ModelingToolkit")
# Pkg.status()

using DifferentialEquations
using Plots
using DiffEqBase, DiffEqOperators, LinearAlgebra
using ModelingToolkit


# Excercise 1

_Local extrema of van der Pol oscillation._ In the van der Pol example
(1.2), you can find the local maxima with $max(y,'local')$, and similarly for minima
with $min$; you can find both minima and maxima at once with $minandmax$. How close is
the first local minimum value (at $t \approx 1.2$) to its asymptotic value for $t \to \infty$? Likewise for the first local maximum (at $t \approx 3.2$)?

Note, that in Julia I didn't see any way to make the minimum/maximum functions work like that, hence the ones I built, at least till I learn of a better way.


## From example 1.2

Van der Pol equation: $0.3u'' -(1-u^2)u' + u = 0$

Borrowing the code the examples Git Repo refered to in the _DifferentialEquations.jl_ docs (look for the Van der Pol examples):
https://github.com/SciML/DiffEqProblemLibrary.jl/blob/master/src/ode/ode_simple_nonlinear_prob.jl

We rewrite the Second Order Function as a system of First Order Functions (per Excercise 1.5):

$$ \frac{dy}{dt} = \mu ((1-x^2)y - x)$$

$$ \frac{dx}{dt} = y$$

Where $\mu = \frac 1 {0.3}$

so:


In [ ]:
@parameters t μ
@variables x(t) y(t)
D = Differential(t)
eqs = [D(y) ~ μ*((1-x^2)*y - x),
       D(x) ~ y]
de = ODESystem(eqs; name=:van_der_pol)
van = ODEFunction(de, [y, x], [μ], jac=true)
u0 = [0.0; 1.0]
vspan = (0.0, 20.0)
μ = 1/0.3
prob_ode_vanderpol = ODEProblem(van, u0, vspan, [μ])
vsol = solve(prob_ode_vanderpol, reltol=1e-8, abstol=1e-8)
plot(vsol, vars=(0, 2))

And get the minimas. First Load my utility routines.

In [ ]:
include("localminmax.jl")

In [ ]:
sigdigits = 6

result = [v[2] for v in vsol.u]
mins = localmins(result)
last_min = nothing
for cur_min in mins
    print("t = ", round(vsol.t[cur_min[1]], sigdigits=sigdigits),
          " u = ", round(cur_min[2], sigdigits=sigdigits))
    if last_min != nothing
        println("   Diff to last = ", round(vsol.t[cur_min[1]] - last_min, sigdigits=sigdigits))
    else
        println("")
    end
    last_min = vsol.t[cur_min[1]]
end
println("")

maxs = localmaxs(result)
last_max = nothing
for cur_max in maxs
    print("t = ", round(vsol.t[cur_max[1]], sigdigits=sigdigits),
          " u = ", round(cur_max[2], sigdigits=sigdigits))
    if last_max != nothing
        println("   Diff to last = ", round(vsol.t[cur_max[1]] - last_max, sigdigits=sigdigits))
    else
        println("")
    end
    last_max = vsol.t[cur_max[1]]
end
println("")



The minima at $t \approx 1.2: u'' = -1.93805$ which is $0.08017 = 3.97%$ off from the minima at $t = 5.2397: u'' = -2.01822$ which closely matches the following minima. However the maxima at $t \approx 3.2: u''=2.01831$ which has already converged to what we see in the following maxima to within 4 significant digits.


# Exercise 1.2

_Classification of ODE problems._ Classify the following ODE problems
according to the FLASHI scheme.

1. $y' = sin(t) − y, t \in [0, 100], y(0) = 1$
    * FLaSHI
2. $y' = sin(t) − y^3, t \in [0, 100], y(0) = 1$
    * FlaSHI
3. (Nonlinear pendulum equation) $y'' = −sin(y), t \in [0, 10], y(0) = y(10) = 2$
    * fLASHI
4. (Advection-diffusion equation) $0.02y'' + y' + y = 0, t \in [0, 1], y(0) = 0, y(1) = 1$
    * fLASHI
5. (Airy equation) $0.02y'' − ty = 0, t \in [−5, 5], y(−5) = 1, y(5) = 0$
    * fLaSHi
6. (Harmonic oscillator) $u' = v, v' = −u, t \in [0, 100], u(0) = 1, v(0) = 0$
    * FLAsHI
7. $u' = u^2v, v' = −uv^2, t \in [0, 2], u(0) = 1, v(0) = 0$
    * FlAsHI
8. (Bessel equation) $t^2y'' + ty' + (t^2 − 4)y = 0, t \in [0, 8], y(0) = 0, y(8) = 1$
    * flaSHI
9. $0.1y'' + yy' = y, t \in [−1, 1], y(−1) = −2, y(1) = 1$
    * flASHi
10. (Lotka–Volterra equations) $u' = u(1−v), v' = v(u−1), t \in [0, 10], u(0) = v(0) = 1$
    * FlAsHi
11. (Blasius equation) $y''' + 0.5yy'' = 0, t \in [0, 10], y(0) = y'(0) = 0, y'(10) = 1$
    * flASHI

# Exercise 1.3

_Airy equation._ Plot the solution $y$ of the problem of Exercise 1.2(5)
and report its maximum value. Letting $k$ denote the coefficient $0.02$, do the same
with $k = 0.002$. Now make a plot of $max(y)$ as a function of $k$ for the values $k =
0.001, 0.002, . . . , 0.039, 0.040$. (We shall explore such effects in Chapter 6.)

(Airy equation) $0.02y'' − ty = 0, t \in [−5, 5], y(−5) = 1, y(5) = 0$

Rewriting for Julia package: $y'' = ty/0.02, t \in [-5, 5], y(-5) = 1, y(5) = 0$

And parameterizing the multiple constant: $y'' - ty/k$

And then reducing to a _First Order System of Equations_:

Let: $y' = x$

so: $y'' = x'$

therefore: $y'' = x' = ty/k$


In [ ]:
# Reformulating this with the macros...
@parameters t k off
@variables x(t) y(t)
D = Differential(t)
airy_eqs = [D(y) ~ (t * x + off) / k,
            D(x) ~ y]
# The following for Fig 7.5
# airy_eqs = [D(x) ~ (sin(t) * y + off) / k,
#             D(y) ~ x]
airy_de = ODESystem(airy_eqs; name=:airy)
airy = ODEFunction(airy_de, [x, y], [k, off])
function airy_bc!(residual, u, p, t)
    # Boundary Conditions: u0 = [0.0; 1.0]
    residual[1] = u[1][1]
    residual[end] = u[end][1] - 1.0
#     residual[end] = u[end][1]
end
aspan = (-5.0, 5.0)
# aspan = (-4π, 4π)
aparam = 0.02
# aparam = 0.003
aoff = 0.0
# aoff = 1.0
initial_guess = [1.0, 0.0]
prob_ode_airy = TwoPointBVProblem(airy, airy_bc!, initial_guess, aspan, [aparam, aoff])
asol = solve(prob_ode_airy, MIRK4(), dt=0.05)
plot(asol, vars=(0, 2))

This clearly doesn't match Fig 7.2 on page 78, but I was able to match Fig 7.5 on page 81 (commented code), so I am confident that I have it formulated correctly.

So, now use the above to look at how varying the constant $k$ changes it.

In [ ]:
max_list = []
k_range = 0.001:0.001:0.040
for k in k_range
    println("k: ", k)
    prob_ode_airy = TwoPointBVProblem(airy, airy_bc!, initial_guess, aspan, [k, aoff])
    println("Problem defined")
    asol = solve(prob_ode_airy, MIRK4(), dt=0.05)
    println("Problem solved")
    push!(max_list, maximum(asol.u[2]))
    println("k: ", k, " max: ", max_list[end])
end

We can see from looking at the max values above (and the first shots at graphing them) that the first value ($k=0.001$) is several orders of magnitude different from the rest of the values, so it's useful to exclude the first number, or the whole graph is just one "large" point, and a line at almost zero, even when we apply log scaling.

I don't know if this is what is expected here, but I am guessing it is properly not. However, this is what I get, so I will leave it for now and maybe come back to it at a future time.

In [ ]:
plot(k_range[2:end], max_list[2:end])  #, xaxis=:log)

# Exercise 1.4

_Solution with rapid transient._ Plot the solution y of the problem of
Exercise 1.2(9), and give its maximum slope $s = max_{t \in [−1,1]} y'(t)$. Do the same with
the coefficient $0.1$ reduced to $1/20$, $1/40$, and $1/80$.

$ 0.1y''+yy'=y, t\in[-1,1], y(-1)=-2, y(1)=1$

Rewriting this for the DifferentialEquations package:

$ 0.1y'' + yy' = y$

Let $y' = x$, then:

$ 0.1x' + yx = y $

$ 0.1x' = y - yx = y(1 - x)$

$ x' = y(1 - x)/0.1 $

Let $p = 0.1$ and we get:

$ x' = y(1 - x)/p $


In [ ]:
@parameters t4 p4
@variables x4(t4) y4(t4)
D = Differential(t4)
eqs4 = [D(x4) ~ y4*(1 - x4) / p4,
        D(y4) ~ x4]

de4 = ODESystem(eqs4; name=:eqs4)
eq4 = ODEFunction(de4, [x4, y4], [p4])
function eqs4_bc!(residual, u, p, t)
    residual[1] = u[1][1] - p[2]
    residual[end] = u[end][1] - p[3]
end
eq4_span = (-1, 1)
eq4_param = 0.1
eq4_boundary = (-2.0, 1.0)
initial_guess = [1.0, 0.0]
prob_ode_eq4 = TwoPointBVProblem(eq4, eqs4_bc!, initial_guess, eq4_span, [eq4_param, eq4_boundary...])
eq4_sol = solve(prob_ode_eq4, MIRK4(), dt=0.05)
plot(title="Parameter = " * string(eq4_param))
#plot!(eq4_sol, vars=(0, 2))
plot!(eq4_sol)

In [ ]:
println("Max([y', y''] = " * string(maximum(eq4_sol.u)))

In [ ]:
eq4_param = 1/20
prob_ode_eq4 = TwoPointBVProblem(eq4, eqs4_bc!, initial_guess, eq4_span, [eq4_param, eq4_boundary...])
eq4_sol = solve(prob_ode_eq4, MIRK4(), dt=0.05)
plot(title="Parameter = " * string(eq4_param))
plot!(eq4_sol)

In [ ]:
println("Max([y', y''] = " * string(maximum(eq4_sol.u)))

In [ ]:
eq4_param = 1/40
prob_ode_eq4 = TwoPointBVProblem(eq4, eqs4_bc!, initial_guess, eq4_span, [eq4_param, eq4_boundary...])
eq4_sol = solve(prob_ode_eq4, MIRK4(), dt=0.05)
plot(title="Parameter = " * string(eq4_param))
plot!(eq4_sol)

In [ ]:
println("Max([y', y''] = " * string(maximum(eq4_sol.u)))

In [ ]:
eq4_param = 1/80
prob_ode_eq4 = TwoPointBVProblem(eq4, eqs4_bc!, initial_guess, eq4_span, [eq4_param, eq4_boundary...])
eq4_sol = solve(prob_ode_eq4, MIRK4(), dt=0.05)
plot(title="Parameter = " * string(eq4_param))
plot!(eq4_sol)

In [ ]:
println("Max([y', y''] = " * string(maximum(eq4_sol.u)))

I don't know if I am interpreting this right, I will continue and maybe come back to it, but it doesn't look like a rapid transient to me - or maybe that's what is shifting $y'$ from just below zero, to just above zero.

# Exercise 1.5

_Reduction to first-order system._ (a) Show how the IVP of Exercise 1.2(6)
can be rewritten as a second-order scalar IVP involving just the dependent variable u.
What are the initial conditions for this IVP? (b) Conversely, show how the third-order
IVP of Exercise 1.2(11) can be rewritten as a first-order system involving three variables
$u$, $v$, and $w$. Any higher-order ODE problem can be rewritten as a first-order problem
like this, and numerical software for IVPs often requires the problem to be expressed
in first-order form.

I am going to skip this, as we have already been doing this to use the Julia DifferentialEquations package in the excercises above.

# Exercise 1.6

How Chebfun represents functions. Chebfun normally represents solutions
$y(t)$ to ODEs by polynomial approximations, typically with an accuracy of about
10 digits, whose degree $n$ may be quite high. The polynomials can be interpreted as
interpolants through a sufficiently large number $n + 1$ of samples at Chebyshev points
defined by $t_j = cos(\pi j/n), 0 \le j \le n$ for $t \in [−1, 1]$, or linearly transplanted to a
different interval $[a, b]$. (a) Let $y$ be the computed solution of the problem (1.1). Execute
$length(y)$ to find the number $n + 1$ for this function, and $plot(y,'.-')$ to see
the associated Chebyshev points. (b) Do the same for the computed solution of the
van der Pol problem (1.2). Approximately speaking (say, to within 10%), how many
interpolation points are there on average per wavelength?

I am going to skip this one as well. I don't know how the Julia DifferentialEquations package works, and this is quite probably doable in a similar way, but it appears to be sufficiently different that the discussion might not (but probably does) hold. We can probably do this easily enough by looking at $solution.t$.

# Exercise 1.7

How Chebfun represents periodic functions. Chebfun also has a representation
for periodic functions that takes advantage of the periodicity, based on
trigonometric polynomials (i.e., Fourier series) rather than ordinary algebraic polynomials.
Periodic solutions arise naturally in ODEs with periodic coefficients (see Chapter
19). (a) Construct an ordinary chebfun for $f(t) = (1.1 − cos(\pi t))
−1, t \in [−1, 1]$
with the command $chebfun('1/(1.1-cos(pi*x))')$. What is its length? (b) How
does the length change if you use $chebfun('1/(1.1-cos(pi*x))','trig')$? (In an
appropriate limit, the ratio of the two lengths approaches $\pi/2$.)

Again, this is refering to the internals of Chebfun, and Julia's package is different. I may come back to these in the future, but for now I will proceed to chapter 2.